In [ ]:
# First we will import the necessary Library
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import re
import os
import math

# Create a dataframe to scrape news article

In [ ]:
# Parameters:
ndays = 30 #30 by default (start date: today -30 days, end date: today)
time_step = 60  # 60 minutes by default
API_KEY = 'd195cbc01116434d8c32b702d4fe31ef'  # Replace API (50 requests/12 hours)


In [ ]:
!pip install newsapi-python
import yfinance as yf
import pandas as pd
from newsapi import NewsApiClient
from datetime import datetime, timedelta

# Configuration des dates dynamiques
END_DATE = datetime.now().strftime('%Y-%m-%d')  # Aujourd'hui
START_DATE = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')  # Un mois avant

# Paramètres de configuration
NEWS_SYMBOLS = ['btc bitcoin crypto cryptocurrency']  # Symboles des actions à analyser

DATABASE_URI = 'sqlite:///etude_de_cas.db'  # URI de la base de données SQLite

# Initialisation des clients API
newsapi = NewsApiClient(api_key=API_KEY)

# Collecter les données boursières
def collect_news_articles(symbol):
    articles = pd.DataFrame()
    for i in range(29):
        all_articles = newsapi.get_everything(q=symbol,
                                            from_param=(datetime.now() - timedelta(days=i+1)).strftime('%Y-%m-%d'),
                                            to=(datetime.now() - timedelta(days=i)).strftime('%Y-%m-%d'),
                                            language='en',
                                            sort_by='publishedAt')
        articles1 = pd.DataFrame(all_articles['articles'])
        articles = pd.concat([articles, articles1], ignore_index=True)

    return articles

#Articles in dataframes:
for symbol in NEWS_SYMBOLS:
    news_df = collect_news_articles(symbol)
print("La collecte et le stockage des données sont terminés.")


La collecte et le stockage des données sont terminés.


In [ ]:
news_df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'The Sports Daily'}",Gia Nguyen,Best Crypto Sportsbooks For Super Bowl 2024: B...,"Heading into Super Bowl 2024, Bitcoin is up ne...",https://thesportsdaily.com/news/best-crypto-sp...,https://thesportsdaily.com/wp-content/uploads/...,2024-02-11T03:57:34Z,"Heading into Super Bowl 2024, Bitcoin is up ne..."
1,"{'id': None, 'name': 'Decrypt'}",Mathew Di Salvo,"This Week in Coins: Bitcoin and Solana Jump, R...",Both Bitcoin and Solana cleared 11% gains over...,https://decrypt.co/216781/this-week-in-coins-b...,https://cdn.decrypt.co/resize/1024/height/512/...,2024-02-11T00:11:31Z,Illustration by Mitchell Preffer for Decrypt.\...
2,"{'id': None, 'name': 'Biztoc.com'}",investorplace.com,Millionaire-Maker Altcoins: 3 Moonshot Cryptos...,Get in early on the early-stage crypto project...,https://biztoc.com/x/24e7c38331fcb375,https://c.biztoc.com/p/24e7c38331fcb375/s.webp,2024-02-11T23:48:17Z,Get in early on the early-stage crypto project...
3,"{'id': None, 'name': 'Zacjohnson.com'}",Disclaimer and DMCA,"CoinSmart CA Affiliate Program – Payout, Revie...","Welcome to the CoinSmart CA Affiliate Program,...",https://zacjohnson.com/coinsmart-ca-affiliate-...,https://0043d976.rocketcdn.me/wp-content/uploa...,2024-02-11T17:36:46Z,"Welcome to the CoinSmart CA Affiliate Program,..."
4,"{'id': None, 'name': 'Zacjohnson.com'}",Disclaimer and DMCA,"CoinSmart Affiliate Program – Payout, Review a...",Are you looking to monetize your website or so...,https://zacjohnson.com/coinsmart-affiliate-pro...,https://0043d976.rocketcdn.me/wp-content/uploa...,2024-02-11T16:22:53Z,Are you looking to monetize your website or so...
...,...,...,...,...,...,...,...,...
1877,"{'id': None, 'name': 'newsBTC'}",Scott Matherson,Bitcoin Hashrate Hits New All-Time High Amid S...,"Bitcoin’s hash rate, that is the network’s com...",http://www.newsbtc.com/news/bitcoin-hashrate-h...,https://www.newsbtc.com/wp-content/uploads/202...,2024-01-14T11:00:32Z,"Bitcoin’s hash rate, that is the network’s com..."
1878,"{'id': None, 'name': 'Biztoc.com'}",newsbtc.com,"Bitcoin To $34,000? Analyst Predicts Next Move...",Bitcoin had a surprisingly underwhelming price...,https://biztoc.com/x/1d00aefe1af041e2,https://c.biztoc.com/p/1d00aefe1af041e2/s.webp,2024-01-14T09:58:08Z,Bitcoin had a surprisingly underwhelming price...
1879,"{'id': None, 'name': 'newsBTC'}",Opeyemi Sule,"Bitcoin To $34,000? Analyst Predicts Next Move...",Bitcoin had a surprisingly underwhelming price...,http://www.newsbtc.com/news/bitcoin/bitcoin-to...,https://www.newsbtc.com/wp-content/uploads/202...,2024-01-14T09:50:06Z,Bitcoin had a surprisingly underwhelming price...
1880,"{'id': None, 'name': 'newsBTC'}",Christian Encila,"ETF Frenzy: Bitcoin Takes A Dip, Ethereum Goes...","In a week etched in crypto history, the long-a...",http://www.newsbtc.com/news/etf-frenzy-bitcoin...,https://www.newsbtc.com/wp-content/uploads/202...,2024-01-14T08:14:07Z,"In a week etched in crypto history, the long-a..."


# Preparation to Sentiment Analysis

In [ ]:
# Prepare news_df to sentiment analysis
news_df = news_df.rename(columns={'publishedAt': 'date', 'title': 'Title', 'content': 'Text'})
news_df['date'] = pd.to_datetime(news_df['date'])


news_df['year'] = news_df['date'].dt.year
news_df['day'] = news_df['date'].dt.day

# Group columns by 'date' with a X-minute interval(Articles published each X minutes ==> BTC prediction each X minutes)
news_df['date'] = news_df['date'].dt.floor(f'{time_step}T')

# Format 'date' to 'year-month-day-hour-minute'
news_df['date'] = news_df['date'].dt.strftime('%Y-%m-%d %H:%M')

news_df.head()

,source,author,Title,description,url,urlToImage,date,Text,year,day
0,"{'id': None, 'name': 'The Sports Daily'}",Gia Nguyen,Best Crypto Sportsbooks For Super Bowl 2024: B...,"Heading into Super Bowl 2024, Bitcoin is up ne...",https://thesportsdaily.com/news/best-crypto-sp...,https://thesportsdaily.com/wp-content/uploads/...,2024-02-11 03:00,"Heading into Super Bowl 2024, Bitcoin is up ne...",2024,11
1,"{'id': None, 'name': 'Decrypt'}",Mathew Di Salvo,"This Week in Coins: Bitcoin and Solana Jump, R...",Both Bitcoin and Solana cleared 11% gains over...,https://decrypt.co/216781/this-week-in-coins-b...,https://cdn.decrypt.co/resize/1024/height/512/...,2024-02-11 00:00,Illustration by Mitchell Preffer for Decrypt.\...,2024,11
2,"{'id': None, 'name': 'Biztoc.com'}",investorplace.com,Millionaire-Maker Altcoins: 3 Moonshot Cryptos...,Get in early on the early-stage crypto project...,https://biztoc.com/x/24e7c38331fcb375,https://c.biztoc.com/p/24e7c38331fcb375/s.webp,2024-02-11 23:00,Get in early on the early-stage crypto project...,2024,11
3,"{'id': None, 'name': 'Zacjohnson.com'}",Disclaimer and DMCA,"CoinSmart CA Affiliate Program – Payout, Revie...","Welcome to the CoinSmart CA Affiliate Program,...",https://zacjohnson.com/coinsmart-ca-affiliate-...,https://0043d976.rocketcdn.me/wp-content/uploa...,2024-02-11 17:00,"Welcome to the CoinSmart CA Affiliate Program,...",2024,11
4,"{'id': None, 'name': 'Zacjohnson.com'}",Disclaimer and DMCA,"CoinSmart Affiliate Program – Payout, Review a...",Are you looking to monetize your website or so...,https://zacjohnson.com/coinsmart-affiliate-pro...,https://0043d976.rocketcdn.me/wp-content/uploa...,2024-02-11 16:00,Are you looking to monetize your website or so...,2024,11


# Save to csv

In [ ]:
news_df.to_csv('news_datasets_text.csv', encoding='utf-8-sig',index= False)
